Name: Hamdi Erkut

I hereby declare that I observed the honour code of the university when preparing the homework.

## Solution to 16.3-5 and 18

## Programming Homework 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
import numpy as np
import math as math
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        ##T.append(row)
        ## T is filled with float instead of strings
        T.append(np.array(row).astype(np.float))
T = np.array(T)
##print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
##display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
##print(T[letter2idx['q']][letter2idx['u']])
##display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
##for c,p in zip(alphabet,T[letter2idx['a']]):
##    print(c,p)
##sumTmp = 0.0
##for c,p in zip(alphabet,T[letter2idx['.']]):
##    sumTmp += float(p)
##print (sumTmp)

# 1)

In [3]:
## function to normalize a vector
def normalizeVector(V):
    return V/np.sum(V)
## end of the normalization of a vector

## function to normalize a matrix
def normalizeMatrix(M):
    N = []
    for c in letter2idx:
        N.append(normalizeVector(M[letter2idx[c]]))
    return N
## end of the normalization of a matrix

##function to creata string containing N character
def createAStringWithoutEndChar(n):
    prevChar = '.'
    result = "."
    for i in range(1, n):
        tmp = np.random.choice(alphabet,p = normalizeVector(T[letter2idx[prevChar]]))
        result += tmp
        prevChar = tmp
    print (result)
## end of the function


# 2)

In [4]:
##function to creata string containing N character whose first and last character is defined
def createAStringWithStartAndEndChar(n,firstChar,lastChar):
    prevChar = firstChar
    result = firstChar
    for i in range(2, n):
        tmp = np.random.choice(alphabet, p = normalizeMatrix(T*np.power(T.transpose(),n-i))[letter2idx[prevChar]])
        result += tmp
        prevChar = tmp
    result += lastChar
    return result
## end of the function

# 3)

In [9]:
##function to creata string containing N character
def createMostProbableStringWithoutEndChar(n,firstChar):
    prevChar = firstChar
    result = ''
    for i in range(1, n):
        tmp = alphabet[np.argmax(normalizeVector(T[letter2idx[prevChar]]))]
        result += tmp
        prevChar = tmp
    return result
## end of the function

##function to creata string containing N character whose last character is 
def createMostProbableStringWithStartAndEndChar(n,firstChar,lastChar):
    prevChar = firstChar
    result = ''
    for i in range(2, n):
        mostProbableCharIndex = np.argmax(normalizeMatrix(T*np.power(T.transpose(),n-i))[letter2idx[prevChar]])
        tmp = alphabet[mostProbableCharIndex]
        result += tmp
        prevChar = tmp
    result += lastChar
    return result
## end of the function

def runTests(strArr):
    for string in strArr:
        i = 0
        creationProcessStarted = 0
        size = 0
        firstChar = '.'
        lastChar = '.'
        result = ''
        while i <= len(string)-1:
            if string[i] == '_' :
                if(bool(creationProcessStarted) == 0):
                    creationProcessStarted = 1
                    if i-1 >= 0:
                        firstChar = string[i-1]
                        size+=1
                    else:
                        firstChar = '.'
                        size+=1
                size+=1
                i+=1
            else:
                if (creationProcessStarted == 1):
                    lastChar = string[i]
                    size+=1
                    result += createMostProbableStringWithStartAndEndChar(size,firstChar,lastChar)
                    size = 0
                    firstChar = '.'
                    lastChar = '.'
                    creationProcessStarted = False
                else:
                    result += string[i]
                i+=1
        if(creationProcessStarted == 1):
            result += createMostProbableStringWithoutEndChar(size,firstChar)
        print ("\n"+result+"\n")
## end of tests

runTests(test_strings)


tht.brern.fox.


tusst.tno.tofbe.tnsw.re.


illat..tarhtno.terreitg


q.s.t.derz.devert.s..der.s..

